<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/UTXO_data_analysis_Task_4_with_extended_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime

## Import Data from Google Drive and Data Wrangling

In [ ]:
# Importing drive method from colab for accessing google drive
from google.colab import drive

# Mounting drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
df_2010=pd.read_csv('/content/drive/My Drive/joint_2010.csv')
df_2010.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
df_2011=pd.read_csv('/content/drive/My Drive/joint_2011.csv')
df_2011.tail()

,num,value,block_date,spent_block_date
1048570,415429,8.496683e+06,8/6/2011,8/8/2011
1048571,415965,1.200000e+08,8/6/2011,8/8/2011
1048572,416087,1.700000e+07,8/6/2011,8/8/2011
1048573,416508,5.884410e+07,8/6/2011,8/8/2011
1048574,420271,1.222000e+09,8/6/2011,8/8/2011


In [ ]:
merge = [df_2010, df_2011]
df = pd.concat(merge)
df.head()

,num,value,block_date,spent_block_date
0,21553,5.000000e+09,1/9/2009,1/12/2009
1,1,5.000000e+09,1/9/2009,NaN
2,2,5.000000e+09,1/9/2009,NaN
3,3,5.000000e+09,1/9/2009,NaN
4,4,5.000000e+09,1/9/2009,NaN


In [ ]:
# Generate the UTXO value in bitcoin unit, which = $value/10^{8}$
df['UTXO'] = df['value']*10**(-8)
df = df.reset_index()
df = df.drop(['value','index','num'], axis = 1)
df.tail()

,block_date,spent_block_date,UTXO
1186094,8/6/2011,8/8/2011,0.084967
1186095,8/6/2011,8/8/2011,1.200000
1186096,8/6/2011,8/8/2011,0.170000
1186097,8/6/2011,8/8/2011,0.588441
1186098,8/6/2011,8/8/2011,12.220000


In [ ]:
# Change the block_date and spent_block_date to datatime object
df['block_date'] = pd.to_datetime(df['block_date'], format='%m/%d/%Y')
df['spent_block_date'] = pd.to_datetime(df['spent_block_date'], format='%m/%d/%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186099 entries, 0 to 1186098
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   block_date        1186099 non-null  datetime64[ns]
 1   spent_block_date  1164547 non-null  datetime64[ns]
 2   UTXO              1186099 non-null  float64       
dtypes: datetime64[ns](2), float64(1)
memory usage: 27.1 MB


#Task 4: Calculate the Distribution for UTXO (Bitcoin Age Distribution for the Bitcoin that are still Alive)

In [ ]:
from datetime import datetime
### calculate the age of each UTXO untill 2009-01-09
duration=pd.date_range(start='2009-01-09', end='2020-11-16')
size=np.size(duration)

### Generate the UTXO dataframe with zeros

In [ ]:
df_UTXO = pd.DataFrame(np.zeros((size, 1)))
df_UTXO.columns=['date']
df_UTXO['date']=duration
df_UTXO.tail()

,date
4325,2020-11-12
4326,2020-11-13
4327,2020-11-14
4328,2020-11-15
4329,2020-11-16


In [ ]:
df_UTXO.insert(1,'< 1d',0)
df_UTXO.insert(2,'1d ~ 1m',0)
df_UTXO.insert(3,'1m ~ 1q',0)
df_UTXO.insert(4,'1q ~ 6m',0)
df_UTXO.insert(5,'6m ~ 1y',0)
df_UTXO.insert(6,'1y ~ 2y',0)
df_UTXO.insert(7,'2y ~ 3y',0)
df_UTXO.insert(8,'3y ~ 4y',0)
df_UTXO.insert(9,'4y ~ 5y',0)
df_UTXO.insert(10, '> 5y', 0)
df_UTXO.head()

,date,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
0,2009-01-09,0,0,0,0,0,0,0,0,0,0
1,2009-01-10,0,0,0,0,0,0,0,0,0,0
2,2009-01-11,0,0,0,0,0,0,0,0,0,0
3,2009-01-12,0,0,0,0,0,0,0,0,0,0
4,2009-01-13,0,0,0,0,0,0,0,0,0,0


## We first try to calculate the UTXO first row and check whether it is correct. If so, we can do a for loop to finish the rest

In [ ]:
now=duration[0]

In [ ]:
col=now.strftime("%Y-%m-%d")

We first drop the UTXO that has been spent "now"=2009-01-09"

In [ ]:
df = df.drop(df[df['spent_block_date']<=now].index)
df.head()

,block_date,spent_block_date,UTXO
0,2009-01-09,2009-01-12,50.0
1,2009-01-09,NaT,50.0
2,2009-01-09,NaT,50.0
3,2009-01-09,NaT,50.0
4,2009-01-09,NaT,50.0


In [ ]:
df[col] = now
df.head()

,block_date,spent_block_date,UTXO,2009-01-09
0,2009-01-09,2009-01-12,50.0,2009-01-09
1,2009-01-09,NaT,50.0,2009-01-09
2,2009-01-09,NaT,50.0,2009-01-09
3,2009-01-09,NaT,50.0,2009-01-09
4,2009-01-09,NaT,50.0,2009-01-09


In [ ]:
df[col] = df[col] - df['block_date']
df.tail()

,block_date,spent_block_date,UTXO,2009-01-09
1186094,2011-08-06,2011-08-08,0.084967,-939 days
1186095,2011-08-06,2011-08-08,1.200000,-939 days
1186096,2011-08-06,2011-08-08,0.170000,-939 days
1186097,2011-08-06,2011-08-08,0.588441,-939 days
1186098,2011-08-06,2011-08-08,12.220000,-939 days


In [ ]:
df[col] = df[col].map(lambda x:x.days)
df.head()

,block_date,spent_block_date,UTXO,2009-01-09
0,2009-01-09,2009-01-12,50.0,0
1,2009-01-09,NaT,50.0,0
2,2009-01-09,NaT,50.0,0
3,2009-01-09,NaT,50.0,0
4,2009-01-09,NaT,50.0,0


In [ ]:
df[col] = pd.to_numeric(df[col])

In [ ]:
df.tail()

,block_date,spent_block_date,UTXO,2009-01-09
1186094,2011-08-06,2011-08-08,0.084967,-939
1186095,2011-08-06,2011-08-08,1.200000,-939
1186096,2011-08-06,2011-08-08,0.170000,-939
1186097,2011-08-06,2011-08-08,0.588441,-939
1186098,2011-08-06,2011-08-08,12.220000,-939


In [ ]:
df.head()

,block_date,spent_block_date,UTXO,2009-01-09
0,2009-01-09,2009-01-12,50.0,0
1,2009-01-09,NaT,50.0,0
2,2009-01-09,NaT,50.0,0
3,2009-01-09,NaT,50.0,0
4,2009-01-09,NaT,50.0,0


In [ ]:
df.columns

Index(['block_date', 'spent_block_date', 'UTXO', '2009-01-09'], dtype='object')

In [ ]:
df_UTXO.loc[(df_UTXO['date']==col),'< 1d']=df[df[col]==0]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'1d ~ 1m']=df[(df[col]>0) & (df[col]<30)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'1m ~ 1q']=df[(df[col]>=30) & (df[col]<91)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'1q ~ 6m']=df[(df[col]>=91) & (df[col]<182)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'6m ~ 1y']=df[(df[col]>=182) & (df[col]<365)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'1y ~ 2y']=df[(df[col]>=365) & (df[col]<365*2)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'2y ~ 3y']=df[(df[col]>=365*2) & (df[col]<365*3)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'3y ~ 4y']=df[(df[col]>=365*3) & (df[col]<365*4)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'4y ~ 5y']=df[(df[col]>=365*4) & (df[col]<365*5)]['UTXO'].sum()
df_UTXO.loc[(df_UTXO['date']==col),'> 5y']=df[(df[col]>=365*5)]['UTXO'].sum()
df_UTXO.tail()

,date,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
4325,2020-11-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4326,2020-11-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4327,2020-11-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4328,2020-11-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4329,2020-11-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
### remember to drop the temporal column for calculation each time after the new row is appended
df=df.drop([col],axis=1)

## Now, we can iterate on duration=pd.date_range(start='2009-01-09', end='2010-12-31')
size=np.size(duration) to get the result for all date

Hint: the final pandas will have 722 more columns

Hint: please remember to drop the rows that has been spent before "now", the current calculating column to reduce data size

In [ ]:
for i in range(size):
  now=duration[i]
  col=now.strftime("%Y-%m-%d")
  df=df.drop(df[df['spent_block_date']<=now].index)
  df[col]=now
  df[col]=df[col]-df['block_date']
  df[col]=df[col].map(lambda x:x.days)
  df[col]=pd.to_numeric(df[col])
  df_UTXO.loc[(df_UTXO['date']==col),'< 1d']=df[df[col]==0]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'1d ~ 1m']=df[(df[col]>0) & (df[col]<30)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'1m ~ 1q']=df[(df[col]>=30) & (df[col]<91)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'1q ~ 6m']=df[(df[col]>=91) & (df[col]<182)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'6m ~ 1y']=df[(df[col]>=182) & (df[col]<365)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'1y ~ 2y']=df[(df[col]>=365) & (df[col]<365*2)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'2y ~ 3y']=df[(df[col]>=365*2) & (df[col]<365*3)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'3y ~ 4y']=df[(df[col]>=365*3) & (df[col]<365*4)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'4y ~ 5y']=df[(df[col]>=365*4) & (df[col]<365*5)]['UTXO'].sum()
  df_UTXO.loc[(df_UTXO['date']==col),'> 5y']=df[(df[col]>=365*5)]['UTXO'].sum()
  df=df.drop([col],axis=1)

In [ ]:
df_UTXO.tail()

,date,< 1d,1d ~ 1m,1m ~ 1q,1q ~ 6m,6m ~ 1y,1y ~ 2y,2y ~ 3y,3y ~ 4y,4y ~ 5y,> 5y
717,2010-12-27,58657.846641,621139.851557,1.015283e+06,538667.439603,615739.80,1361813.75,0.0,0.0,0.0,0.0
718,2010-12-28,30373.383958,649963.364240,1.005467e+06,551420.609713,615439.80,1366063.75,0.0,0.0,0.0,0.0
719,2010-12-29,86582.816836,594611.775616,1.008916e+06,552183.239713,613065.18,1371013.75,0.0,0.0,0.0,0.0
720,2010-12-30,38565.832386,631745.420067,1.024782e+06,551924.529713,612425.18,1375113.75,0.0,0.0,0.0,0.0
721,2010-12-31,29209.053351,650457.379102,1.006546e+06,565527.930675,611919.19,1378613.75,0.0,0.0,0.0,0.0


In [ ]:
import plotly.offline as py     
import plotly.graph_objects as go

trace0 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["< 1d"], name = '< 1d')
trace1 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["1d ~ 1m"], name = '1d ~ 1m')
trace2 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["1m ~ 1q"], name = '1m ~ 1q')
trace3 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["1q ~ 6m"], name = '1q ~ 6m')
trace4 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["6m ~ 1y"], name = '6m ~ 1y')
trace5 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["1y ~ 2y"], name = '1y ~ 2y')
trace6 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["2y ~ 3y"], name = '2y ~ 3y')
trace7 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["3y ~ 4y"], name = '3y ~ 4y')
trace8 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["4y ~ 5y"], name = '4y ~ 5y')
trace9 = go.Scatter(x = df_UTXO["date"], y = df_UTXO["> 5y"], name = '> 5y')

layout = go.Layout(title="UTXO Real-time Life Expectancy Distribution", xaxis = dict(title = "date"), yaxis = dict(title = "Value with UTXO's Life Expectancy") ) 
data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9]

fig = go.Figure(layout = layout, data = data) 
fig.update_yaxes(type="log")

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

py.iplot(fig)